## Iterative Adv Training

### For iter in Range(20):
-       Step 1: Generate training adv examples to 1) overwrite shot_adv.png 2) create coco_perturb_train.json
-       Step 2: Mix downsampled normal training data with perturbed training data (4:1) and create coco_adv_mix.json
-       Step 3: Evaluate training Acc on all clean+adv examples (9 metrics)
-       Step 4: Gerenate testing adv examples .... 

In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import pickle

In [2]:
import os
os.chdir('..')

In [3]:
!pwd

/home/l/liny/yizhe/code/detectron2-1


In [6]:

for i in range(20):
    if i == 0:
        weight_last_epoch = 'output/rcnn_bet365/model_final.pth'
    else:
        weight_last_epoch = 'output/adv_%s/model_final.pth'%(str(i - 1))
    output_this_epoch = 'output/adv_%s'%(str(i))
    weight_this_epoch = output_this_epoch + '/model_final.pth'
    
    '''generate training, testing adv examples'''
    !python run_DAG.py --test False --cfg-path configs/faster_rcnn_bet365.yaml \
                                               --weights-path {weight_last_epoch} \
                                               --results-save-path coco_instances_ignore_train.json 

    !python run_DAG.py --test True --cfg-path configs/faster_rcnn_bet365.yaml \
                                              --weights-path {weight_last_epoch} \
                                              --results-save-path coco_instances_ignore_test.json 
        
    #mix normal with training
    !python detectron2_1/mix_normal_adv.py --normal-path data/benign_data/coco_train.json \
                                           --adv-path data/benign_data/coco_perturbgt_train.json \
                                           --save-path data/benign_data/coco_adv_mix.json 

    #retrain 
    !python train_net.py --config-file configs/faster_rcnn_adv.yaml \
                                       OUTPUT_DIR {output_this_epoch} \
                                       MODEL.WEIGHTS {weight_last_epoch} \
    
    # evaluate training acc
    !python train_net.py --eval-only --config-file configs/faster_rcnn_adv_eval_train.yaml \
                                                   MODEL.WEIGHTS {weight_this_epoch} \
                                                   OUTPUT_DIR {output_this_epoch} # Path to trained checkpoint
    
    gt_coco_path = 'data/benign_data/coco_adv_mix.json' # here the ground-truth is for the original-sized image
    results_coco_path =  output_this_epoch + '/coco_instances_results.json' # here the prediction is for the resized image
    coco_gt = COCO(gt_coco_path)
    coco_dt = coco_gt.loadRes(results_coco_path)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    with open(output_this_epoch + '/all_trainacc.pkl', 'wb') as f:
        pickle.dump(coco_eval, f)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.catIds = [1]
    with open(output_this_epoch + '/input_trainacc.pkl', 'wb') as f:
        pickle.dump(coco_eval, f)
        
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.catIds = [2]
    with open(output_this_epoch + '/logo_trainacc.pkl', 'wb') as f:
        pickle.dump(coco_eval, f)
        
    # evaluate testing acc
    !python train_net.py --eval-only --config-file configs/faster_rcnn_adv_eval_test.yaml \
                                     MODEL.WEIGHTS {weight_this_epoch} \
                                     OUTPUT_DIR {output_this_epoch} # Path to trained checkpoint
    
    gt_coco_path = 'data/benign_data/coco_perturbgt_test.json' 
    results_coco_path =  output_this_epoch + '/coco_instances_results.json' 
    coco_gt = COCO(gt_coco_path)
    coco_dt = coco_gt.loadRes(results_coco_path)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    with open(output_this_epoch + '/all_testacc.pkl', 'wb') as f:
        pickle.dump(coco_eval, f)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.catIds = [1]
    with open(output_this_epoch + '/input_testacc.pkl', 'wb') as f:
        pickle.dump(coco_eval, f)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.catIds = [2]
    with open(output_this_epoch + '/logo_testacc.pkl', 'wb') as f:
        pickle.dump(coco_eval, f)
    
        

loading annotations into memory...
Done (t=10.52s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.63s)
creating index...
index created!
Command Line Args: Namespace(config_file='configs/faster_rcnn_adv_eval_test.yaml', dist_url='tcp://127.0.0.1:49923', eval_only=True, machine_rank=0, num_gpus=1, num_machines=1, opts=['MODEL.WEIGHTS', 'output/adv_0/model_final.pth', 'OUTPUT_DIR', 'output/adv_0'], resume=False)
[09/09 10:55:14 detectron2]: Rank of current process: 0. World size: 1
[09/09 10:55:15 detectron2]: Environment info:
----------------------  -------------------------------------------------------------------------------------------
sys.platform            linux
Python                  3.7.7 (default, Mar 23 2020, 22:36:06) [GCC 7.3.0]
numpy                   1.18.1
detectron2              0.1.3 @/home/l/liny/anaconda3/envs/mypy37/lib/python3.7/site-packages/detectron2
Compiler                GCC 5.4
CUDA compiler           CUDA 9.2
detectron2 arch fl

[09/09 10:55:19 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[09/09 10:55:20 d2.data.datasets.coco]: Loaded 1579 images in COCO format from data/benign_data/coco_perturbgt_test.json
[09/09 10:55:20 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|    box     | 2284         |    logo    | 1462         |
|            |              |            |              |
|   total    | 3746         |            |              |
[09/09 10:55:20 d2.data.common]: Serializing 1579 elements to byte tensors and concatenating them all ...
[09/09 10:55:20 d2.data.common]: Serialized dataset takes 0.45 MiB
[09/09 10:55:20 d2.evaluation.evaluator]: Start inference on 1579 images
[09/09 10:55:25 d2.evaluation.evaluator]: Inference done 11/1579. 0.0440 s / img. ETA=0:10:40
[09/09 10:55:30 d2.evaluation.evaluator]: Inference done 19/1579. 0.0447 s / img. ETA=0:13:59
[09/09 10:55:36 d2.evaluation.evaluator]: Inference done 28/1579. 0.0450 

[09/09 11:02:00 d2.evaluation.evaluator]: Inference done 714/1579. 0.0444 s / img. ETA=0:08:04
[09/09 11:02:05 d2.evaluation.evaluator]: Inference done 723/1579. 0.0443 s / img. ETA=0:07:59
[09/09 11:02:10 d2.evaluation.evaluator]: Inference done 733/1579. 0.0443 s / img. ETA=0:07:53
[09/09 11:02:16 d2.evaluation.evaluator]: Inference done 738/1579. 0.0443 s / img. ETA=0:07:54
[09/09 11:02:21 d2.evaluation.evaluator]: Inference done 752/1579. 0.0444 s / img. ETA=0:07:43
[09/09 11:02:27 d2.evaluation.evaluator]: Inference done 769/1579. 0.0444 s / img. ETA=0:07:29
[09/09 11:02:33 d2.evaluation.evaluator]: Inference done 771/1579. 0.0444 s / img. ETA=0:07:33
[09/09 11:02:39 d2.evaluation.evaluator]: Inference done 793/1579. 0.0443 s / img. ETA=0:07:15
[09/09 11:02:44 d2.evaluation.evaluator]: Inference done 813/1579. 0.0443 s / img. ETA=0:06:58
[09/09 11:02:49 d2.evaluation.evaluator]: Inference done 836/1579. 0.0444 s / img. ETA=0:06:39
[09/09 11:02:54 d2.evaluation.evaluator]: Inferenc

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Command Line Args: Namespace(config_file='configs/faster_rcnn_adv_eval_test.yaml', dist_url='tcp://127.0.0.1:49923', eval_only=True, machine_rank=0, num_gpus=1, num_machines=1, opts=['MODEL.WEIGHTS', 'output/adv_1/model_final.pth', 'OUTPUT_DIR', 'output/adv_1'], resume=False)
[09/09 11:06:45 detectron2]: Rank of current process: 0. World size: 1
[09/09 11:06:47 detectron2]: Environment info:
----------------------  -------------------------------------------------------------------------------------------
sys.platform            linux
Python                  3.7.7 (default, Mar 23 2020, 22:36:06) [GCC 7.3.0]
numpy                   1.18.1
detectron2              0.1.3 @/home/l/liny/anaconda3/envs/mypy37/lib/python3.7/site-packages/detectron2
Compiler                GCC 5.4
CUDA compiler           CUDA 9.2
detectron2 arch fla

[09/09 11:06:48 detectron2]: Full config saved to output/adv_1/config.yaml
[09/09 11:06:48 d2.utils.env]: Using a generated random seed 49164121
[09/09 11:06:52 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_si

        (1): ROIAlign(output_size=(7, 7), spatial_scale=0.125, sampling_ratio=0, aligned=True)
        (2): ROIAlign(output_size=(7, 7), spatial_scale=0.0625, sampling_ratio=0, aligned=True)
        (3): ROIAlign(output_size=(7, 7), spatial_scale=0.03125, sampling_ratio=0, aligned=True)
      )
    )
    (box_head): FastRCNNConvFCHead(
      (fc1): Linear(in_features=12544, out_features=1024, bias=True)
      (fc2): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (box_predictor): FastRCNNOutputLayers(
      (cls_score): Linear(in_features=1024, out_features=3, bias=True)
      (bbox_pred): Linear(in_features=1024, out_features=8, bias=True)
    )
  )
)
[09/09 11:06:52 fvcore.common.checkpoint]: Loading checkpoint from output/adv_1/model_final.pth
Traceback (most recent call last):
  File "train_net.py", line 151, in <module>
    args=(args,),
  File "/home/l/liny/anaconda3/envs/mypy37/lib/python3.7/site-packages/detectron2/engine/launch.py", line 57, in launch
    main

FileNotFoundError: [Errno 2] No such file or directory: 'output/adv_2/coco_instances_results.json'

In [7]:
with open('./output/adv_1/logo_testacc.pkl', 'rb') as f:
    logo_testacc = pickle.load(f)
    
with open('./output/adv_1/input_testacc.pkl', 'rb') as f:
    input_testacc = pickle.load(f)
    
with open('./output/adv_1/all_testacc.pkl', 'rb') as f:
    all_testacc = pickle.load(f)

In [8]:
with open('./output/adv_1/logo_trainacc.pkl', 'rb') as f:
    logo_trainacc = pickle.load(f)
    
with open('./output/adv_1/input_trainacc.pkl', 'rb') as f:
    input_trainacc = pickle.load(f)
    
with open('./output/adv_1/all_trainacc.pkl', 'rb') as f:
    all_trainacc = pickle.load(f)

In [14]:
logo_testacc.evaluate()
logo_testacc.accumulate()
logo_testacc.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.24s).
Accumulating evaluation results...
DONE (t=0.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100